In [1]:
!apt install unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 175 kB of archives.
After this operation, 386 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.2 [175 kB]
Fetched 175 kB in 0s (815 kB/s)
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 16763 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-26ubuntu3.2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking unzip (6.0-26ubuntu3.2) ...
7Progress: [ 40%] [#######################.......................

In [2]:
!curl -L -o include-interpolated-cv-300p-random-rotate.zip https://www.kaggle.com/api/v1/datasets/download/riceyy/include-interpolated-cv-300p-random-rotate
!curl -L -o include-interpolated-cv-300p-random-crop.zip https://www.kaggle.com/api/v1/datasets/download/riceyy/include-interpolated-cv-300p-random-crop
!curl -L -o include-interpolated-300p.zip https://www.kaggle.com/api/v1/datasets/download/naneet1/include-interpolated-300p
!curl -L -o include-interpolated-cv-300p-double-madness.zip https://www.kaggle.com/api/v1/datasets/download/riceyy/include-interpolated-cv-300p-double-madness

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  251M  100  251M    0     0  23.4M      0  0:00:10  0:00:10 --:--:-- 27.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  246M  100  246M    0     0  22.8M      0  0:00:10  0:00:10 --:--:-- 27.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  134M  100  134M    0     0  20.6M      0  0:00:06  0:00:06 --:--:-- 25.9M
  % Total    % Received % Xferd  Average Speed   Tim

In [3]:
!unzip include-interpolated-cv-300p-random-rotate.zip -d /home/include-interpolated-cv-300p-random-rotate
!unzip include-interpolated-cv-300p-random-crop.zip -d /home/include-interpolated-cv-300p-random-crop
!unzip include-interpolated-300p.zip -d /home/include-interpolated-300p
!unzip include-interpolated-cv-300p-double-madness.zip -d /home/include-interpolated-cv-300p-double-madness

Archive:  include-interpolated-cv-300p-random-rotate.zip
  inflating: Test_data/tensor_0.pt   
  inflating: Test_data/tensor_1.pt   
  inflating: Test_data/tensor_10.pt  
  inflating: Test_data/tensor_11.pt  
  inflating: Test_data/tensor_12.pt  
  inflating: Test_data/tensor_13.pt  
  inflating: Test_data/tensor_14.pt  
  inflating: Test_data/tensor_2.pt   
  inflating: Test_data/tensor_3.pt   
  inflating: Test_data/tensor_4.pt   
  inflating: Test_data/tensor_5.pt   
  inflating: Test_data/tensor_6.pt   
  inflating: Test_data/tensor_7.pt   
  inflating: Test_data/tensor_8.pt   
  inflating: Test_data/tensor_9.pt   
  inflating: Train_data/tensor_0.pt  
  inflating: Train_data/tensor_1.pt  
  inflating: Train_data/tensor_10.pt  
  inflating: Train_data/tensor_11.pt  
  inflating: Train_data/tensor_12.pt  
  inflating: Train_data/tensor_13.pt  
  inflating: Train_data/tensor_14.pt  
  inflating: Train_data/tensor_15.pt  
  inflating: Train_data/tensor_16.pt  
  inflating: Train_data/

In [5]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision import models
from torchvision.transforms.functional import to_tensor
from torch.amp import autocast, GradScaler

import os
import numpy as np
import subprocess
from PIL import Image
import shutil
import torch.nn.functional as F

# %matplotlib inline
# from matplotlib import pyplot as plt
# import matplotlib.gridspec as gridspec
# os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [6]:
device = 'cuda'

In [13]:
def read_pickle(path) -> torch.tensor:
    return torch.load(path)

def pickle_files_path_list(path) -> list:
    path_list = []
    for top, dirs, files in os.walk(path):
        for nm in files:
            path_list.append(os.path.join(top, nm))
    return path_list

def preload_tensors(path_list, device):
    data = []
    for path in path_list:
        tensor = torch.load(path)  # Load tensor from disk
        video, label = tensor['video'].to(device), tensor['label'].to(device)
        data.append((video.to(torch.float16), label))
    return data

In [14]:
import torch

def save_checkpoint(state, filename="checkpoint.pth"):
    """Save model and optimizer state to a file."""
    torch.save(state, filename)

def load_checkpoint(filename="checkpoint.pth"):
    """Load model and optimizer state from a file."""
    checkpoint = torch.load(filename)
    return checkpoint

In [15]:
class Trainer:
    def __init__(self, model, device, optimizer, loss_fn, scheduler, save=False):
        self.device = device
        self.scaler = GradScaler()  # Automatic Mixed Precision
        self.optimizer = optimizer
        self.scheduler = scheduler  # Add scheduler
        self.loss_fn = loss_fn
        self.model = model
        self.save = save

    def train_step_pickle(self, epoch, preloaded_data):
        self.model.train()
        train_loss, total_correct, total_samples = 0, 0, 0

        for X, y in preloaded_data:
            # Forward pass with AMP
            with autocast('cuda'):
                y_pred = self.model(X)
                loss = self.loss_fn(y_pred, y)
                train_loss += loss.item()

            # Backward pass with scaling
            self.scaler.scale(loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
            self.optimizer.zero_grad()

            # Accuracy computation
            y_pred_class = torch.argmax(y_pred, dim=1)
            total_correct += (y_pred_class == y).sum().item()
            total_samples += y.size(0)

        acc = total_correct * 100 / total_samples
        train_loss /= len(preloaded_data)
        print(f"Epoch: {epoch} | Train Loss: {train_loss:.4f} | Accuracy: {acc:.2f}")

        # Step scheduler if it's not ReduceLROnPlateau
        if not isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            self.scheduler.step()

    def test_step_pickle(self, epoch,preloaded_data):
        self.model.eval()
        test_loss, acc = 0, 0
        total_correct = 0
        total_samples = 0

        with torch.no_grad():
            for X, y in preloaded_data:
                # Use AMP for inference
                with autocast('cuda'):
                    y_pred = self.model(X)
                    loss = self.loss_fn(y_pred, y)
                    test_loss += loss.item()

                # Compute predictions and accuracy
                y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
                total_correct += (y_pred_class == y).sum().item()
                total_samples += y.size(0)

            acc = total_correct * 100 / total_samples
            test_loss = test_loss / len(preloaded_data)
            print(f"Epoch: {epoch} | Test Loss: {test_loss:.4f} | Accuracy: {acc:.2f}")
            
            # Step scheduler if ReduceLROnPlateau
            if isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                self.scheduler.step(test_loss)  # Use validation loss as the metric
            
            if self.save:
                print(f"Saving model at epoch {epoch}...")
                save_checkpoint({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'best_metric': acc
                }, filename=f"resnet_pretrained_acc={acc:.2f}_loss={loss:.4f}_{epoch=}_real.pth")
            print("************************")


In [20]:
train_preloaded = []
train_path = ['/home/include-interpolated-cv-300p-random-crop/Train_data','/home/include-interpolated-300p/ISL_pickle/Train_data',
              '/home/include-interpolated-cv-300p-double-madness/Train_data','/home/include-interpolated-cv-300p-random-rotate/Train_data']

for path in train_path:
    train_preloaded.extend(preload_tensors(pickle_files_path_list(path), device='cuda:0'))


test_preloaded_all = []
test_path_all = ['/home/include-interpolated-cv-300p-random-crop/Test_data','/home/include-interpolated-300p/ISL_pickle/Test_data',
                '/home/include-interpolated-cv-300p-double-madness/Test_data','/home/include-interpolated-cv-300p-random-rotate/Test_data']

for path in test_path_all:
    test_preloaded_all.extend(preload_tensors(pickle_files_path_list(path), device='cuda:0'))

test_preloaded_unaug = preload_tensors(pickle_files_path_list('/home/include-interpolated-300p/ISL_pickle/Test_data'), device='cuda:0')



# train_preloaded = preload_tensors(pickle_files_path_list('/kaggle/input/include-interpolated-cv-300p-random-crop/Train_data'), device='cuda:0')
# test_preloaded = preload_tensors(pickle_files_path_list('/kaggle/input/include-interpolated-cv-300p-random-crop/Test_data'), device='cuda:0')

/tmp/ipykernel_547/4016792477.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(path)  # Load tensor from disk


In [21]:
len(train_preloaded), len(test_preloaded_all), len(test_preloaded_unaug)

(204, 60, 15)

In [22]:
import torch
import torch.nn as nn
from torchvision import models

# CNN Model
class r3d(nn.Module):
    def __init__(self, num_classes):
        super(r3d, self).__init__()

        # CNN (ResNet - 3D)
        self.cnn = models.video.r3d_18(pretrained=True)  # Pretrained 3D ResNet
        self.cnn.fc = nn.Identity()  # Remove final classification layer

        # Fully connected layer for classification
        self.fc = nn.Linear(512, num_classes)  # r3d_18 final layer outputs 512 features

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape sequence as depth for 3D CNN
        x = x.permute(0, 2, 1, 3, 4)  # [Batch, Channels, Depth (Seq_Len), Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch, 512, 1, 1, 1]
        features = features.view(features.size(0), -1)  # Flatten: [Batch, 512]

        # Pass through the fully connected layer
        output = self.fc(features)  # [Batch, Num_Classes]
        return output

In [23]:
import torch
import torch.nn as nn
from torchvision import models

# CNN + GRU Model with Dropouts
class r18gru(nn.Module):
    def __init__(self, num_classes, dropout):
        super(r18gru, self).__init__()

        # Pre-trained CNN (e.g., ResNet)
        self.cnn = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Remove the final classification layer

        # Dropout after CNN features
        self.cnn_dropout = nn.Dropout(p=dropout)

        # GRU for temporal modeling
        self.gru = nn.GRU(input_size=512, hidden_size=256, num_layers=2, batch_first=True, dropout=dropout)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

        # Dropout before final FC layer
        self.fc_dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape input for CNN processing
        x = x.view(batch_size * seq_len, c, h, w)  # Combine Batch and Sequence dimensions: [Batch * Sequence, Channels, Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch * Sequence, 512, 1, 1] (ResNet-18 output)
        features = features.view(features.size(0), -1)  # Flatten: [Batch * Sequence, 512]

        # Apply CNN dropout
        features = self.cnn_dropout(features)

        # Reshape features back to sequence format for GRU
        features = features.view(batch_size, seq_len, -1)  # [Batch, Sequence, Features]

        # Pass the sequence of features through the GRU
        gru_out, _ = self.gru(features)  # [Batch, Sequence, Hidden_Size]

        # Take the output of the last time step
        final_output = gru_out[:, -1, :]  # [Batch, Hidden_Size]

        # Apply dropout before the fully connected layer
        final_output = self.fc_dropout(final_output)

        # Pass through the fully connected layer
        output = self.fc(final_output)  # [Batch, Num_Classes]
        return output

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# CNN + LSTM Model
class r18lstm(nn.Module):
    def __init__(self, num_classes):
        super(r18lstm, self).__init__()

        # Pre-trained CNN (e.g., ResNet)
        self.cnn = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Remove the final classification layer

        # LSTM for temporal modeling
        self.lstm = nn.LSTM(input_size=512, hidden_size=256, num_layers=2, batch_first=True)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape input for CNN processing
        x = x.view(batch_size * seq_len, c, h, w)  # Combine Batch and Sequence dimensions: [Batch * Sequence, Channels, Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch * Sequence, 512, 1, 1] (ResNet-18 output)
        features = features.view(features.size(0), -1)  # Flatten: [Batch * Sequence, 512]

        # Reshape features back to sequence format for LSTM
        features = features.view(batch_size, seq_len, -1)  # [Batch, Sequence, Features]

        # Pass the sequence of features through the LSTM
        lstm_out, _ = self.lstm(features)  # [Batch, Sequence, Hidden_Size]

        # Take the output of the last time step
        final_output = lstm_out[:, -1, :]  # [Batch, Hidden_Size]

        # Pass through the fully connected layer
        output = self.fc(final_output)  # [Batch, Num_Classes]
        return output

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# CNN + GRU Model with Dropouts
class mv3gru(nn.Module):
    def __init__(self, num_classes, dropout):
        super(mv3gru, self).__init__()

        # Pre-trained CNN (e.g., ResNet)
        self.cnn = models.mobilenet_v3_large(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Remove the final classification layer

        # Dropout after CNN features
        self.cnn_dropout = nn.Dropout(p=dropout)

        # GRU for temporal modeling
        self.gru = nn.GRU(input_size=960, hidden_size=256, num_layers=2, batch_first=True, dropout=dropout)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

        # Dropout before final FC layer
        self.fc_dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape input for CNN processing
        x = x.view(batch_size * seq_len, c, h, w)  # Combine Batch and Sequence dimensions: [Batch * Sequence, Channels, Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch * Sequence, 512, 1, 1] (ResNet-18 output)
        features = features.view(features.size(0), -1)  # Flatten: [Batch * Sequence, 512]

        # Apply CNN dropout
        features = self.cnn_dropout(features)

        # Reshape features back to sequence format for GRU
        features = features.view(batch_size, seq_len, -1)  # [Batch, Sequence, Features]

        # Pass the sequence of features through the GRU
        gru_out, _ = self.gru(features)  # [Batch, Sequence, Hidden_Size]

        # Take the output of the last time step
        final_output = gru_out[:, -1, :]  # [Batch, Hidden_Size]

        # Apply dropout before the fully connected layer
        final_output = self.fc_dropout(final_output)

        # Pass through the fully connected layer
        output = self.fc(final_output)  # [Batch, Num_Classes]
        return output

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# CNN + GRU Model with Dropouts
class r50gru(nn.Module):
    def __init__(self, num_classes, dropout):
        super(r50gru, self).__init__()

        # Pre-trained CNN (e.g., ResNet)
        self.cnn = models.resnet50(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Remove the final classification layer

        # Dropout after CNN features
        self.cnn_dropout = nn.Dropout(p=dropout)

        # GRU for temporal modeling
        self.gru = nn.GRU(input_size=2048, hidden_size=256, num_layers=2, batch_first=True, dropout=dropout)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

        # Dropout before final FC layer
        self.fc_dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape input for CNN processing
        x = x.view(batch_size * seq_len, c, h, w)  # Combine Batch and Sequence dimensions: [Batch * Sequence, Channels, Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch * Sequence, 512, 1, 1] (ResNet-18 output)
        features = features.view(features.size(0), -1)  # Flatten: [Batch * Sequence, 512]

        # Apply CNN dropout
        features = self.cnn_dropout(features)

        # Reshape features back to sequence format for GRU
        features = features.view(batch_size, seq_len, -1)  # [Batch, Sequence, Features]

        # Pass the sequence of features through the GRU
        gru_out, _ = self.gru(features)  # [Batch, Sequence, Hidden_Size]

        # Take the output of the last time step
        final_output = gru_out[:, -1, :]  # [Batch, Hidden_Size]

        # Apply dropout before the fully connected layer
        final_output = self.fc_dropout(final_output)

        # Pass through the fully connected layer
        output = self.fc(final_output)  # [Batch, Num_Classes]
        return output

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# CNN + GRU Model with Dropouts
class r50lstm(nn.Module):
    def __init__(self, num_classes, dropout):
        super(r50lstm, self).__init__()

        # Pre-trained CNN (e.g., ResNet)
        self.cnn = models.resnet50(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Remove the final classification layer

        # Dropout after CNN features
        self.cnn_dropout = nn.Dropout(p=dropout)

        # GRU for temporal modeling
        self.gru = nn.LSTM(input_size=2048, hidden_size=256, num_layers=2, batch_first=True, dropout=dropout)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

        # Dropout before final FC layer
        self.fc_dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        """
        Forward pass for the model.

        Args:
        - x: Input tensor with shape [Batch, Sequence, Channels, Height, Width]

        Returns:
        - output: Model predictions with shape [Batch, Num_Classes]
        """
        batch_size, seq_len, c, h, w = x.size()  # x: [Batch, Sequence, Channels, Height, Width]

        # Reshape input for CNN processing
        x = x.view(batch_size * seq_len, c, h, w)  # Combine Batch and Sequence dimensions: [Batch * Sequence, Channels, Height, Width]

        # Extract features using CNN
        features = self.cnn(x)  # [Batch * Sequence, 512, 1, 1] (ResNet-18 output)
        features = features.view(features.size(0), -1)  # Flatten: [Batch * Sequence, 512]

        # Apply CNN dropout
        features = self.cnn_dropout(features)

        # Reshape features back to sequence format for GRU
        features = features.view(batch_size, seq_len, -1)  # [Batch, Sequence, Features]

        # Pass the sequence of features through the GRU
        gru_out, _ = self.gru(features)  # [Batch, Sequence, Hidden_Size]

        # Take the output of the last time step
        final_output = gru_out[:, -1, :]  # [Batch, Hidden_Size]

        # Apply dropout before the fully connected layer
        final_output = self.fc_dropout(final_output)

        # Pass through the fully connected layer
        output = self.fc(final_output)  # [Batch, Num_Classes]
        return output

In [17]:
# shutil.rmtree('/kaggle/working')

OSError: [Errno 16] Device or resource busy: '/kaggle/working'

In [30]:
# model = r18gru(num_classes=50,dropout=0.1).to(device)
model = r3d(50).to(device)
# model = r50lstm(50).to(device, dropout=0.1)
# model = r50gru(50).to(device)
# model = r18lstm(50).to(device)
# model = mv3gru(50).to(device)
# model = nn.DataParallel(model)

In [31]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Define the scheduler without the verbose parameter
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=5,
    min_lr=1e-6  # Set a minimum learning rate
)


# Define the loss function
loss_fn = nn.CrossEntropyLoss()


In [32]:
trainer = Trainer(device=device,optimizer=optimizer,loss_fn=loss_fn,save=True,scheduler=scheduler, model=model)
epochs = 100
for epoch in range(epochs):
    print("Train:")
    trainer.train_step_pickle(epoch=epoch, preloaded_data=train_preloaded)
    print("Test all:")
    trainer.test_step_pickle(epoch=epoch, preloaded_data=test_preloaded_all)

Train:
Epoch: 0 | Train Loss: 2.7071 | Accuracy: 48.76
Test all:
Epoch: 0 | Test Loss: 2.2607 | Accuracy: 55.84
Saving model at epoch 0...
************************
Train:
Epoch: 1 | Train Loss: 1.0202 | Accuracy: 92.79
Test all:
Epoch: 1 | Test Loss: 1.7306 | Accuracy: 63.96
Saving model at epoch 1...
************************
Train:
Epoch: 2 | Train Loss: 0.3309 | Accuracy: 99.28
Test all:
Epoch: 2 | Test Loss: 1.4053 | Accuracy: 65.26
Saving model at epoch 2...
************************
Train:


KeyboardInterrupt: 

In [ ]:
# Plotting Confusion Matrix
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
from sklearn.metrics import confusion_matrix

y_true = []
y_preds = []

model.eval()
with torch.no_grad():
  for X, y in test_preloaded:
    X, y = X.to(device).float(), y.to(device)


    y_pred = model(X)
    y_pred_class = y_pred.argmax(dim=1)

    y_true.extend(y.cpu().numpy())
    y_preds.extend(y_pred_class.cpu().numpy())
plt.figure(figsize=(1000, 1000))
cm = confusion_matrix(y_true, y_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(50))
disp.plot(cmap="viridis")  # Customize colormap if needed
plt.xticks(fontsize=6, rotation=45, ha="right")
plt.yticks(fontsize=6)
plt.show()
import seaborn as sns
def cm_disp(cm):
    mask = np.zeros_like(cm)
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(cm, mask=mask, vmax=.3, square=True,  cmap="YlGnBu")
    plt.show()
cm_disp(cm)

In [ ]:
# Plotting Confusion Matrix
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
from sklearn.metrics import confusion_matrix

y_true = []
y_preds = []

model.eval()
with torch.no_grad():
  for X, y in train_preloaded:
    X, y = X.to(device).float(), y.to(device)


    y_pred = model(X)
    y_pred_class = y_pred.argmax(dim=1)

    y_true.extend(y.cpu().numpy())
    y_preds.extend(y_pred_class.cpu().numpy())
plt.figure(figsize=(1000, 1000))
cm = confusion_matrix(y_true, y_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(50))
disp.plot(cmap="viridis")  # Customize colormap if needed
plt.xticks(fontsize=6, rotation=45, ha="right")
plt.yticks(fontsize=6)
plt.show()
import seaborn as sns
def cm_disp(cm):
    mask = np.zeros_like(cm)
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(cm, mask=mask, vmax=.3, square=True,  cmap="YlGnBu")
    plt.show()
cm_disp(cm)

In [ ]:
from IPython.display import FileLink
FileLink('resnet_pretrained_acc=88.74_loss=0.2813_epoch=30_real.pth')

In [9]:
# Load checkpoint for model1 (r18gru)
model1 = r18gru(num_classes=50, dropout=0.1).to(device)
model1 = nn.DataParallel(model1)  # Distribute model across GPUs

checkpoint1 = torch.load('/kaggle/input/models/resnet_pretrained_GRU_acc88.74_loss0.2813_epoch30_real.pth')
model1.load_state_dict(checkpoint1['model_state_dict'])

# Load checkpoint for model2 (r3d_18)
model2 = r3d(50).to(device)
model2 = nn.DataParallel(model2)

checkpoint2 = torch.load('/kaggle/input/models/r3d_18_pretrained_acc89.18_loss0.7433_epoch15_real.pth')
model2.load_state_dict(checkpoint2['model_state_dict'])


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s] 
<ipython-input-9-7ecaffe87109>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possi

<All keys matched successfully>

In [41]:
y_pred = []
y_true = []
model1.eval()
model2.eval()
with torch.no_grad():
    for X, y in test_preloaded:
        y_pred_batch = 0
        # Convert the labels and predictions to numpy arrays
        y_true_batch = y.tolist()
        #y_pred_batch += model1(X.to(torch.float32))
        y_pred_batch += model2(X.to(torch.float32))
        #y_pred_batch /= 2
        # Concatenate the current batch predictions and labels
        y_true.extend(y_true_batch)  # Wrap arrays in a tuple
        y_pred.extend(y_pred_batch.argmax(dim=1).tolist())

In [42]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
types= ['micro', 'macro', 'weighted']
for ty in types:
    print(ty)
    # Assuming y_true and y_pred are your ground truth and predictions
    precision = precision_score(y_true, y_pred, average=ty)  # Use 'binary' for binary classification
    recall = recall_score(y_true, y_pred, average=ty)        # Use 'macro' or 'micro' for multiclass if needed
    f1 = f1_score(y_true, y_pred, average=ty) # 'weighted' handles class imbalance
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print("***************")
print(f"Acc = {accuracy_score(y_true, y_pred):.4f}")


micro
F1 Score: 0.2641
Precision: 0.2641
Recall: 0.2641
***************
macro
F1 Score: 0.2501
Precision: 0.3475
Recall: 0.2630
***************
weighted
F1 Score: 0.2588
Precision: 0.3621
Recall: 0.2641
***************
Acc = 0.2641


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
